# Predictions

For calculating the predictions regarding the i) number of infections, ii) number of hospitalisations, iii) number of Intensive Care Units and iv) number of deaths, we'll use ARMA - Autoregressive Moving Average 

In [1]:
from imports_functions import *
%matplotlib inline

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
db1 = pd.read_csv('covid-19-sample.csv')

In [3]:
db1 =clean_data(db1)

We'll treat the data as a time series. For that, we shall aplply evolution function and set the date as index:

In [4]:
bydate=evolution(db1).set_index('date')

## Infections

We should train an ARMA model for a given p and q, for a given set of days. After several experiments and simulations, one found that we minimise the root square mean error if we train the model with the whole dataset, and predict on forward:

In [5]:
infections = training_models(bydate, 'infections')

In [6]:
pred_infections = pred_infections(infections)

/home/ffreire/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'


In [7]:
pred_infections

,pred_infections
2021-06-11,3436.980030
2021-06-12,3383.025769
2021-06-13,3347.409818
2021-06-14,3330.748341
2021-06-15,3328.881073
2021-06-16,3341.760262
2021-06-17,3369.143182


## Hospitalisations

In [8]:
hosp = training_models(bydate, 'hospitalisations')

In [9]:
pred_hosp = pred_hosp(hosp)

/home/ffreire/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
/home/ffreire/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/home/ffreire/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [10]:
pred_hosp

,pred_hosp
2021-06-11,159.873646
2021-06-12,157.179157
2021-06-13,158.815545
2021-06-14,162.838239
2021-06-15,167.128339
2021-06-16,179.841826
2021-06-17,190.712059


## Intensive Care Units

In [11]:
uci = training_models(bydate, 'icu')

In [12]:
pred_uci = pred_uci(uci)

/home/ffreire/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'


## Deaths

In [13]:
deaths = training_models(bydate, 'deaths')

In [14]:
deaths

,ave_7_num_dead
date,
2020-02-23,0.000000
2020-02-24,0.000000
2020-02-25,0.000000
2020-02-26,0.000000
2020-02-27,0.000000
...,...
2021-06-06,17.428571
2021-06-07,16.000000
2021-06-08,16.571429


In [15]:
pred_deaths = pred_deaths(deaths)

/home/ffreire/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'


In [16]:
pred_deaths

,pred_deaths
2021-06-11,13.016844
2021-06-12,12.667172
2021-06-13,13.765634
2021-06-14,14.791417
2021-06-15,16.457542
2021-06-16,18.820792
2021-06-17,21.751402


In [17]:
predictions = pd.concat([pred_infections, pred_hosp, pred_uci, pred_deaths], axis = 1)

In [ ]:
predictions.to_csv('predictions.csv', index=True)